In [1]:
import scipy.stats as st
import numpy as np
import pandas as pd
from scipy import optimize

In [2]:
qnorm = st.norm.ppf
pnorm = st.norm.cdf

In [3]:
123123

123123

\begin{equation*}
cdf(x) = Pr(X \leq x)
\end{equation*}

## 波动性

In [4]:
##条件极大似然估计
data = pd.read_csv('C:/Users/17800/Desktop/python/sp500.csv')
data = data.iloc[0:20]

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/17800/Desktop/python/sp500.csv'

In [ ]:
def a(n,beta,dt=data):
    if n ==1:
        return np.sqrt(np.var(dt))
    else:
        return dt.iloc[n-1]-beta[0]-beta[1]*dt.iloc[n-2]

def sigma_2(n,beta,dt=data):
    if n == 1:
        return np.var(dt)
    else:
        return beta[2] + beta[3]*sigma_2(n-1,beta,dt=data)+beta[4]*a(n-1,beta,dt=data)**2

In [ ]:
def loss(beta,dt=data):
    #s1 = pd.Series([i for i in range(2,len(dt)+1)])
    #ss = s1.apply(lambda t:sigma_2(t,beta))
    #sr = s1.apply(lambda t:(dt.iloc[t-1]-beta[0]-beta[1]*dt.iloc[t-2])**2)
    aa = 0
    for t in range(2,len(dt)+1):
        aa += np.log(sigma_2(t,beta))+(dt.iloc[t-1]-beta[0]-beta[1]*dt.iloc[t-2])**2/sigma_2(t,beta)
    return -aa

In [ ]:
bnds = ((-10,10),(-10,10),(0,1),(0.0001,1),(0.0001,1))
optimize.minimize(loss,(0.1,0.1,0.1,0.1,0.1),bounds=bnds)

### 波动率估计

#### GARCH(1,1)

In [5]:
p_omega = 0.000002
p_alpha = 0.03
p_beta = 0.95
p_s_0 = 0.014
# 给定当前的波动率为s_0,则s_t^2的估计为
def V(t,omega=p_omega,alpha=p_alpha,beta=p_beta,s_0=p_s_0):
    V_L = omega/(1-alpha-beta)
    return V_L + (alpha+beta)**t*(s_0**2-V_L)

#持续期限为T的期权的波动率
def sigma(T,omega=p_omega,alpha=p_alpha,beta=p_beta,s_0=p_s_0):
    a = -np.log(alpha+beta)
    V_L = omega/(1-alpha-beta)
    return np.sqrt(252*((V_L+((1-np.exp(-a*T))/(a*T))*(V(0)-V(T)))))

In [6]:
np.sqrt(V(50))

0.011617241035993148

In [7]:
sigma(10)

0.17090053968985033

### WCDR

In [8]:
# worst case default rate
# 违约率在时间段T内不超过X的概率
def WCDR(T,X,rho,PD):
    return pnorm((qnorm(PD)+np.sqrt(rho)*qnorm(X))/(np.sqrt(1-rho)))

In [9]:
WCDR(15,0.9999,0.0826,0.06725)

0.32759001848758695

### Vasicek模型中PD和rho的估计

In [10]:
# 违约概率
data = pd.Series([0.02,0.04,0.07,0.12,0.06,0.05,0.08,0.14,0.1,0.02,0.03,0.02,0.06,0.07,0.09,0.06,0.11])

In [13]:
# 定义损失函数
def gfunc(x,r,PD):
    return np.sqrt((1-r)/r)*np.exp(0.5*(st.norm.ppf(x)**2-((np.sqrt(1-r)*st.norm.ppf(x)-st.norm.ppf(PD))/np.sqrt(r))**2))

def loss(beta,dt=data):
    return -dt.apply(lambda t:gfunc(t,beta[0],beta[1])).prod()

In [14]:
#极大似然估计结果
optimize.minimize(loss,(0.1,0.1)).x

array([0.08262462, 0.06725271])

## VaR

In [36]:
st.norm.ppf(0.95)

1.6448536269514722

### 自相关修正的VaR

In [109]:
a = 0
rho = 0.1
T = 10
for i in range(1,T+1):
    for j in range(1,T+1):
        a += (0.16)**np.abs(i-j)
a

13.356009075281241

### VaR单边test

假定置信水平为$p=(1-X)$，观测期为n天，实际有m天损失超过-VaR(p)。如果$\frac{m}{n} \geq p$,我们想检验VaR是不是定的太低。
\begin{equation*} 
H_0:\frac{m}{n} = p  \\
H_1:\frac{m}{n}  \geq p
\end{equation*}  
在H1成立的情况下，Z(超过VaR(p)的次数)
\begin{equation*}
    Z \sim Binomial(n,p)
\end{equation*}  
所以出现比m次更极端的概率为
\begin{equation*}
p-value=1-Binomial(m-1,n,p)
\end{equation*}


In [8]:
1-st.binom.cdf(14,n=800,p=0.01)

0.01675064861961062

### Kupiec Test

- p:VaR的置信水平
- m:例外的次数
- n:总观测此时

In [4]:
def Kup_test(m,n,p):
    LR = -2*np.log(((1-p)**(n-m))*(p**m))+2*np.log((1-m/n)**(n-m)*(m/n)**m)
    return LR

In [5]:
Kup_test(15,800,0.01)

4.920311552403831

![在线编辑Markdown](https://github.com/nymath/Nymath/微信图片_20220505111644.jpg)

<img src="C:/Users/17800/Desktop/微信图片_20220505111644.jpg">

In [10]:
qnorm(1)

inf